In [1]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import time

from backtesting.test import  SMA
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply,crossover

from skopt.plots import plot_objective

C:\Users\kmtk49\Anaconda3\envs\TF2-GPU\lib\site-packages\backtesting\_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
ticker = yf.Ticker("8015.T")
hist = ticker.history(period="6mo")
hist = hist.iloc[:,:5]
hist

,Open,High,Low,Close,Volume
Date,,,,,
2020-05-27,2567.561012,2654.130119,2548.869955,2634.455322,1022200
2020-05-28,2682.658647,2727.910682,2649.211491,2695.447266,1096400
2020-05-29,2659.048887,2714.138320,2589.203355,2685.609863,1668700
2020-06-01,2678.723642,2714.138277,2656.097625,2703.317139,1001900
2020-06-02,2715.122041,2779.065132,2700.365943,2767.260254,767300
...,...,...,...,...,...
2020-11-20,3415.000000,3460.000000,3390.000000,3415.000000,585900
2020-11-24,3500.000000,3600.000000,3500.000000,3565.000000,943100
2020-11-25,3635.000000,3720.000000,3620.000000,3655.000000,1348200


In [3]:
def SMA(values,n):
    return pd.Series(values).rolling(n).mean()

class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()
            
def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

class System(Strategy):
    d_rsi = 7 #30  # Daily RSI lookback periods
    w_rsi = 7 #30  # Weekly
    level = 14 #70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 5) #10
        self.ma20 = self.I(SMA, self.data.Close, 10) #20
        self.ma50 = self.I(SMA, self.data.Close, 15) #50
        self.ma100 = self.I(SMA, self.data.Close, 25) #100
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

class Sma4Cross(Strategy):
    n1 = 50
    n2 = 100
    n_enter = 20
    n_exit = 10
    
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.sma_enter = self.I(SMA, self.data.Close, self.n_enter)
        self.sma_exit = self.I(SMA, self.data.Close, self.n_exit)
        
    def next(self):
        
        if not self.position:
            
            # On upwards trend, if price closes above
            # "entry" MA, go long
            
            # Here, even though the operands are arrays, this
            # works by implicitly comparing the two last values
            if self.sma1 > self.sma2:
                if crossover(self.data.Close, self.sma_enter):
                    self.buy()
                    
            # On downwards trend, if price closes below
            # "entry" MA, go short
            
            else:
                if crossover(self.sma_enter, self.data.Close):
                    self.sell()
        
        # But if we already hold a position and the price
        # closes back below (above) "exit" MA, close the position
        
        else:
            if (self.position.is_long and
                crossover(self.sma_exit, self.data.Close)
                or
                self.position.is_short and
                crossover(self.data.Close, self.sma_exit)):
                
                self.position.close()

In [4]:
'''移動平均クロス'''
bt01 = Backtest(hist,SmaCross,cash = 25_000,commission=0.00125)
stats01 = bt01.run()
stats01

Start                     2020-05-27 00:00:00
End                       2020-11-27 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                        69.6
Equity Final [$]                      29683.2
Equity Peak [$]                       29683.2
Return [%]                             18.733
Buy & Hold Return [%]                 39.4975
Return (Ann.) [%]                      41.363
Volatility (Ann.) [%]                 33.1855
Sharpe Ratio                          1.24642
Sortino Ratio                         2.93612
Calmar Ratio                          4.54994
Max. Drawdown [%]                     -9.0909
Avg. Drawdown [%]                    -4.54824
Max. Drawdown Duration       36 days 00:00:00
Avg. Drawdown Duration       17 days 00:00:00
# Trades                                    5
Win Rate [%]                               80
Best Trade [%]                        14.9507
Worst Trade [%]                      -8.52011
Avg. Trade [%]                    

In [5]:
bt01.plot()

In [6]:
%%time
'''移動平均クロス・決定木サーチ'''

stats_skopt, heatmap, optimize_result = bt01.optimize(
    n1=[10, 100],      # Note: For method="skopt", we
    n2=[20, 200],      # only need interval end-points
    constraint=lambda p: p.n1 < p.n2,
    maximize='Equity Final [$]',
    method='skopt',
    max_tries=200,
    random_state=0,
    return_heatmap=True,
    return_optimization=True)
heatmap.sort_values().iloc[-3:]

Wall time: 16.6 s


n1  n2
47  63    30325.541385
64  70    30622.047557
83  84    32139.700000
Name: Equity Final [$], dtype: float64

In [7]:
ma1 = heatmap.sort_values().iloc[-1:]

N1 = ma1.index.get_level_values('n1').values[0]
N2 = ma1.index.get_level_values('n2').values[0]

class SmaCross2(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = N1
    n2 = N2
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

bt01opt = Backtest(hist,SmaCross2,cash = 25_000,commission=0.00125)
stats01opt = bt01opt.run()
stats01opt

Start                     2020-05-27 00:00:00
End                       2020-11-27 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                        27.2
Equity Final [$]                      32139.7
Equity Peak [$]                       32139.7
Return [%]                            28.5588
Buy & Hold Return [%]                 39.4975
Return (Ann.) [%]                     65.9393
Volatility (Ann.) [%]                 25.1999
Sharpe Ratio                          2.61665
Sortino Ratio                         9.84007
Calmar Ratio                          14.2339
Max. Drawdown [%]                    -4.63257
Avg. Drawdown [%]                     -2.0019
Max. Drawdown Duration        7 days 00:00:00
Avg. Drawdown Duration        5 days 00:00:00
# Trades                                    2
Win Rate [%]                              100
Best Trade [%]                        24.7587
Worst Trade [%]                       5.36558
Avg. Trade [%]                    

In [8]:
bt01opt.plot()

In [9]:
'''RSIストラテジー 400%year_trading'''
bt02 = Backtest(hist, System, commission=.00125)
stats02 = bt02.run()
stats02

Start                     2020-05-27 00:00:00
End                       2020-11-27 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                          12
Equity Final [$]                      10068.5
Equity Peak [$]                       10475.6
Return [%]                           0.685306
Buy & Hold Return [%]                 39.4975
Return (Ann.) [%]                     1.38639
Volatility (Ann.) [%]                 7.60364
Sharpe Ratio                         0.182332
Sortino Ratio                        0.289747
Calmar Ratio                         0.339386
Max. Drawdown [%]                    -4.08499
Avg. Drawdown [%]                    -2.10536
Max. Drawdown Duration       78 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                    1
Win Rate [%]                              100
Best Trade [%]                        0.52475
Worst Trade [%]                       0.52475
Avg. Trade [%]                    

In [10]:
bt02.plot()

In [11]:
%%time
'''RSIストラテジー・決定木サーチ'''

stats_skopt, heatmap, optimize_result = bt02.optimize(
    d_rsi = [7 ,50],#30  # Daily RSI lookback periods
    w_rsi = [7 ,50],#30  # Weekly
    level = [7,100], #70      # only need interval end-points
    constraint=lambda p: p.d_rsi < p.w_rsi < p.level,
    maximize='Equity Final [$]',
    method='skopt',
    max_tries=200,
    random_state=0,
    return_heatmap=True,
    return_optimization=True)
heatmap.sort_values().iloc[-3:]

Wall time: 16.2 s


d_rsi  w_rsi  level
15     34     50       10151.102349
13     22     30       10151.102349
       21     29       10151.102349
Name: Equity Final [$], dtype: float64

In [12]:
rsi2 = heatmap.sort_values().iloc[-1:]

D_rsi = rsi2.index.get_level_values('d_rsi').values[0]
W_rsi = rsi2.index.get_level_values('w_rsi').values[0]
Level = rsi2.index.get_level_values('level').values[0]

In [13]:
class System2(Strategy):
    d_rsi = D_rsi #7 #30  # Daily RSI lookback periods
    w_rsi = W_rsi#7 #30  # Weekly
    level = Level #14 #70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 5) #10
        self.ma20 = self.I(SMA, self.data.Close, 10) #20
        self.ma50 = self.I(SMA, self.data.Close, 15) #50
        self.ma100 = self.I(SMA, self.data.Close, 25) #100
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [14]:
bt02opt = Backtest(hist, System2, commission=.00125)
stats02opt = bt02opt.run()
stats02opt

Start                     2020-05-27 00:00:00
End                       2020-11-27 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                           8
Equity Final [$]                      10151.1
Equity Peak [$]                         10579
Return [%]                            1.51102
Buy & Hold Return [%]                 39.4975
Return (Ann.) [%]                     3.06961
Volatility (Ann.) [%]                 6.88488
Sharpe Ratio                         0.445848
Sortino Ratio                        0.760483
Calmar Ratio                         0.758855
Max. Drawdown [%]                    -4.04505
Avg. Drawdown [%]                    -2.72782
Max. Drawdown Duration       78 days 00:00:00
Avg. Drawdown Duration       40 days 00:00:00
# Trades                                    1
Win Rate [%]                              100
Best Trade [%]                        1.68211
Worst Trade [%]                       1.68211
Avg. Trade [%]                    

In [15]:
bt02opt.plot()

In [16]:
'''4本線クロス'''
bt03 = Backtest(hist, Sma4Cross, commission=.00125)
stats03 = bt03.run()
stats03

Start                     2020-05-27 00:00:00
End                       2020-11-27 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                        16.8
Equity Final [$]                      11258.3
Equity Peak [$]                       11258.3
Return [%]                            12.5826
Buy & Hold Return [%]                 39.4975
Return (Ann.) [%]                     26.9891
Volatility (Ann.) [%]                 17.6483
Sharpe Ratio                          1.52927
Sortino Ratio                         3.99719
Calmar Ratio                          4.31862
Max. Drawdown [%]                    -6.24947
Avg. Drawdown [%]                    -3.02909
Max. Drawdown Duration       14 days 00:00:00
Avg. Drawdown Duration        7 days 00:00:00
# Trades                                    2
Win Rate [%]                               50
Best Trade [%]                        19.2636
Worst Trade [%]                      -5.72158
Avg. Trade [%]                    

In [17]:
bt03.plot()

In [18]:
%%time
'''4本線クロス・決定木サーチ'''

stats_skopt, heatmap, optimize_result = bt03.optimize(
    n1=[10, 100],      # Note: For method="skopt", we
    n2=[20, 200],      # only need interval end-points
    n_enter=[10, 40],
    n_exit=[10, 30],
    constraint=lambda p: p.n_exit < p.n_enter < p.n1 < p.n2,
    maximize='Equity Final [$]',
    method='skopt',
    max_tries=200,
    random_state=0,
    return_heatmap=True,
    return_optimization=True)
heatmap.sort_values().iloc[-3:]

Wall time: 18.2 s


n1  n2  n_enter  n_exit
39  50  33       18        11865.584347
38  74  36       13        11865.584347
63  69  36       24        11865.584347
Name: Equity Final [$], dtype: float64

In [19]:
h1 = heatmap.sort_values().iloc[-1:]

N1 = h1.index.get_level_values('n1').values[0]
N2 = h1.index.get_level_values('n2').values[0]
N_enter = h1.index.get_level_values('n_enter').values[0]
N_exit = h1.index.get_level_values('n_exit').values[0]

class Sma4Cross2(Strategy):
    n1 = N1
    n2 = N2
    n_enter = N_enter
    n_exit = N_exit
    
    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
        self.sma_enter = self.I(SMA, self.data.Close, self.n_enter)
        self.sma_exit = self.I(SMA, self.data.Close, self.n_exit)
        
    def next(self):
        
        if not self.position:
            if self.sma1 > self.sma2:
                if crossover(self.data.Close, self.sma_enter):
                    self.buy()
            
            else:
                if crossover(self.sma_enter, self.data.Close):
                    self.sell()
        
        else:
            if (self.position.is_long and
                crossover(self.sma_exit, self.data.Close)
                or
                self.position.is_short and
                crossover(self.data.Close, self.sma_exit)):
                
                self.position.close()
                

In [20]:
bt03opt = Backtest(hist, Sma4Cross2, commission=.00125)
stats03opt = bt03opt.run()
stats03opt

Start                     2020-05-27 00:00:00
End                       2020-11-27 00:00:00
Duration                    184 days 00:00:00
Exposure Time [%]                        24.8
Equity Final [$]                      11865.6
Equity Peak [$]                       11865.6
Return [%]                            18.6558
Buy & Hold Return [%]                 39.4975
Return (Ann.) [%]                      41.178
Volatility (Ann.) [%]                  20.359
Sharpe Ratio                          2.02259
Sortino Ratio                         6.09043
Calmar Ratio                          11.9691
Max. Drawdown [%]                    -3.44037
Avg. Drawdown [%]                    -2.19699
Max. Drawdown Duration       34 days 00:00:00
Avg. Drawdown Duration       11 days 00:00:00
# Trades                                    2
Win Rate [%]                              100
Best Trade [%]                        19.2636
Worst Trade [%]                       1.02504
Avg. Trade [%]                    

In [22]:
bt03opt.plot()